We show how to solve a capacitated facility location problem (CFLP) in Julia using Simulated Annealing[^1], which is a metaheuristic to approximate global optimization in a large search space for an optimization problem.

In order to keep our example simple we will choose a random walk to decide which facility to open and a simple heuristic to assign facilities to customer. 


## Problem statement

A company must select a subset of potential facility locations to minimize total cost associated with opening facilities and servicing customers. Each costumer has a specific demand and each facility has a fixed opening costs, a specific capacity and service costs based on the distance to customers.

## Mathematical model formulation

In order to have a more mathematical problem description we give a formulation as a mixed interger problem next:

### Sets

- $J=\{1,\ldots,m\}$ set of potential facilities
- $C=\{1,\ldots,n\}$ set of customers

### Parameters

- $c^f_j\in\mathbb{R}⁺$ fix opening cost of facility $j$
- $q_j\in\mathbb{N}^+$ capacity of facility $j$
- $d_i\in\mathbb{N}^+$ demand of customer $i$
- $c^v_{i,j}\in\mathbb{R}^+$ servicing costs from facility $j$ to customer $i$

### Decision variables

- $x_{ij}$, real, demand of customer $i$ served by facility $j$
- $y_j$, binary, 1 if and only if facility $j$ is open

### Objective

$$
\min \sum_j c^f_j \cdot y_j + \sum_{ij} c_{ij}x_{ij}
$$

### Constraints

- (c1) Each client's demand is served:

$$
\sum_j x_{ij} = d_i, \; \forall i \in C
$$

- (c2) A facility can serve a client only if its open:

$$
x_{ij} \leq d_i y_j, \forall i\in C, \forall j \in J
$$

- (c3) A facility can not provide more than its capacity

$$
\sum_i x_{ij} \leq q_j y_j, \forall j \in J
$$

- (c4) There is maximal number of facilities allowed to be opened

$$
\sum_j y_j \leq k, 
$$

[1]: We remark that the Julia [Optim package](https://julianlsolvers.github.io/Optim.jl/stable/) containts a general implementation of the Simulated Annealing algorithm.

### Problem formulation as a Mixed Interger Linear Programm


$$
\begin{array}{lll}
\min & \sum_j c^f_j \cdot y_j + \sum_{ij} c_{ij}x_{ij} &\\
s.t. & \sum_j x_{ij} = d_i & \forall i \in C\\
     & x_{ij} \leq d_i y_j & \forall i\in C, \forall j \in J\\
     & \sum_i x_{ij} \leq q_j y_j & \forall j \in J \\
     & \sum_j y_j \leq k & \\
     & x_{ij}\geq0 & \\
     & y_j\in\{0,1\}&
\end{array}
$$

In [1]:
using Random
using LinearAlgebra

In [2]:
# set seed for reporducibility
Random.seed!(1111);
Random.TaskLocalRNG();

## CFLP data structure

## load I/O adn tests functions

We load some 

In [3]:
include("./SA_DataSturctures.jl")
include("./io.jl")
include("./tests.jl")
include("./SA_Algorithms.jl")

SA_client_facility_assignment (generic function with 5 methods)

## Heuristics 

Because the search space for the assignment variable $x_{ij}$ can become extremely large, we simply use a simple heuristic to compute a valid assignment of clients to facilites.
This is encoded in the function `client_facility_assignment`.

In particular we choosed this design as we could replace its definition in order to perform a combined simmulated annealing [@qin2012combined], which means we have to layers:

1. an outer layer algorithm, which optimizes the facility location decision
2. an inner layer algorithm, which optimizes the demand allocation decision based on the open facility decision

and in each layer an simumalted annealing is used.

In [4]:
function SA_cflp_with_heuristic(
        data::cflp_data, k::Int64, start_temp::Float64 = 10.0, end_temp::Float64 = 1.0, alpha::Float64 = 0.5, max_iter::Int64 = 100)
    """
    TBA
    """
    # generate initial solution 
    current_solution = generate_initial_solution(data, k)
    best_solution = current_solution

    # iterate
    temp = start_temp
    while temp > end_temp
        for _ in 1:max_iter
            # generate a new decision on open facilities
            candidate_solution = generate_candidate_facility(data, current_solution, k)
            
            if  test_cflp_solution(data, candidate_solution, k) && candidate_solution.cost < best_solution.cost 
                best_solution = candidate_solution
            end

            if candidate_solution.cost < current_solution.cost
                current_solution = candidate_solution
            else
                acceptance_prob = exp( (current_solution.cost - candidate_solution.cost) / temp )
                if rand() < acceptance_prob
                    current_solution = candidate_solution
                end
            end
            
        end


        
        temp *= alpha
    end

    if !test_cflp_solution(data, best_solution,k)
        println("did not found a feasible solution")
    end
    
    return  best_solution
end

SA_cflp_with_heuristic (generic function with 5 methods)

## Study examples

In [5]:
# data
demand = [80, 270, 250, 160, 180]
capacity = [500,500,500]
cost_fix = [1000.,1000.,1000.]
cost_var = [4. 6. 9.; 5. 4. 7.; 6. 3. 4.; 8. 5. 3.; 10. 8. 4.]

data = cflp_data(cost_fix, cost_var, capacity, demand)

cflp_data([1000.0, 1000.0, 1000.0], [4.0 6.0 9.0; 5.0 4.0 7.0; … ; 8.0 5.0 3.0; 10.0 8.0 4.0], [500, 500, 500], [80, 270, 250, 160, 180], 3, 5, 87.0, 11.0)

In [6]:
sol = SA_cflp_with_heuristic(data, 2)
print_solution(sol)

Total cost are:5610.0
Open facilities:[2, 3]
Assignment:
customer   1 gets       80.000000 from facility   2
customer   2 gets      270.000000 from facility   2
customer   3 gets      150.000000 from facility   2
customer   3 gets      100.000000 from facility   3
customer   4 gets      160.000000 from facility   3
customer   5 gets      180.000000 from facility   3


In [7]:
using JLD2

d = load("../data/data.jld2")["data"]
dont_print_all = true
data = cflp_data(d["cost_fix"], d["cost_var"], d["capacity"], d["demand"])

k = 9
example2 = SA_cflp_with_heuristic(data, k)
print_solution(example2, dont_print_all)

Total cost are:14704.919999999998
Open facilities:[1, 3, 5, 7, 8, 9, 10, 11, 12]
Assignment:
customer   1 gets       18.000000 from facility   9
customer   2 gets       21.000000 from facility   7
customer   3 gets       12.000000 from facility  11
customer   4 gets       14.000000 from facility   3
customer   5 gets       18.000000 from facility   3


optimal MILP solution
 
Total cost are:14277.130000000001
Open facilities:[1, 3, 5, 6, 8, 9, 10, 11, 13]